# Get_Original_AlphaNet_Data

In [1]:
Alpha_Name = "AlphaNet_Original_Input_12.14"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "5d_ret"

In [2]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm

In [3]:
dir_list = ['Data','Cov','Corr','Return','Std','Zscore','Decay']
for i in dir_list:
    os.mkdir(path+i)

FileExistsError: [Errno 17] File exists: '/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data'

In [3]:
df = pd.read_csv(path+"data.csv")
df

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,5d_ret
0,2015-01-06 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,-0.075710
1,2015-01-06 09:30:00,2,1774.23,1885.20,1753.27,1838.34,1823.031224,6560835.70,7.2662,6.7834,8.1741,-0.106164
2,2015-01-06 09:30:00,4,63.31,64.62,62.18,63.76,63.277989,28095.12,0.7707,3.3496,4.5343,0.006390
3,2015-01-06 09:30:00,5,38.00,38.00,38.00,38.00,37.997172,0.00,0.0000,0.0000,0.0000,0.000000
4,2015-01-06 09:30:00,6,225.64,234.85,223.10,225.00,228.438278,1078647.12,0.4255,8.0562,13.2751,-0.078571
...,...,...,...,...,...,...,...,...,...,...,...,...
5357756,2021-07-01 09:30:00,688700,47.80,54.50,47.03,53.38,50.715300,64336.83,14.5494,19.2222,19.2222,0.153171
5357757,2021-07-01 09:30:00,688777,90.21,95.08,89.32,94.42,93.250464,19001.61,4.6111,4.6288,4.6288,0.053133
5357758,2021-07-01 09:30:00,688788,138.07,149.35,137.11,146.22,145.197985,13572.56,5.1954,7.7977,7.7977,-0.102721
5357759,2021-07-01 09:30:00,688819,44.04,45.64,43.95,45.15,45.026221,47249.01,1.8544,4.6754,4.6754,0.032164


In [12]:
# for ticker in tqdm(df['ticker'].drop_duplicates()):
def test(ticker):
    one_data = df[df['ticker'] == ticker]
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
    return concat_df

In [13]:
from multiprocessing import Pool
import multiprocessing as mp
import time
if __name__=="__main__":
    num_cores = int(mp.cpu_count())
    pool = mp.Pool(num_cores)
    result = {}
    t1 = time.time()
    for ticker in tqdm(df['ticker'].drop_duplicates()):
        result[ticker] = (pool.apply_async(test, args=(ticker,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)
final_df

100%|██████████| 4427/4427 [00:00<00:00, 298725.57it/s]

并行执行时间： 40


,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,open,...,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,5d_ret
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,1126.20,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.005735
2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,4.3668,1105.60,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.001433
2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,3.4269,1101.33,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.029915
2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,2.8375,1058.70,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,-0.021739
2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,5.0564,1056.57,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.049075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30 09:30:00,605339,20.38,24.45,20.38,24.45,24.423600,4472.57,43.9929,0.7040,0.7040,26.90,...,44.75,45.44,44.10,44.99,44.862500,43863.72,0.6938,6.9045,6.9045,-0.075876
2021-07-01 09:30:00,605339,26.90,26.90,26.90,26.90,26.900000,1373.52,10.0204,0.2162,0.2162,29.59,...,44.81,44.88,42.70,43.18,43.469700,72397.12,-4.0231,11.3958,11.3958,-0.043207
2021-06-30 09:30:00,688314,100.20,105.00,86.78,92.05,92.899300,106963.24,430.8535,90.5821,90.5821,94.00,...,115.11,115.78,110.21,112.00,112.765600,6286.46,-4.1014,5.3237,5.3237,-0.081182


In [14]:
final_df.columns = [str(i) for i in range(final_df.shape[1])]
final_df.rename(columns={str(final_df.shape[1] - 1):'target'}, inplace=True)
final_df.reset_index(inplace=True)
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
final_df

,timestamp,ticker,0,1,2,3,4,5,6,7,...,261,262,263,264,265,266,267,268,269,target
0,2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.005735
1,2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.001433
2,2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.029915
3,2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,-0.021739
4,2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.049075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230281,2021-06-30 09:30:00,605339,20.38,24.45,20.38,24.45,24.423600,4472.57,43.9929,0.7040,...,44.75,45.44,44.10,44.99,44.862500,43863.72,0.6938,6.9045,6.9045,-0.075876
5230282,2021-07-01 09:30:00,605339,26.90,26.90,26.90,26.90,26.900000,1373.52,10.0204,0.2162,...,44.81,44.88,42.70,43.18,43.469700,72397.12,-4.0231,11.3958,11.3958,-0.043207
5230283,2021-06-30 09:30:00,688314,100.20,105.00,86.78,92.05,92.899300,106963.24,430.8535,90.5821,...,115.11,115.78,110.21,112.00,112.765600,6286.46,-4.1014,5.3237,5.3237,-0.081182
5230284,2021-07-01 09:30:00,688314,94.00,106.12,83.00,102.50,92.323600,80295.81,11.3525,67.9987,...,112.32,112.32,103.50,105.64,107.011300,9925.98,-5.6786,8.4058,8.4058,0.043254


In [15]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
time_list = [20150101,20150301,20150601,20150901,
             20160101,20160301,20160601,20160901,
             20170101,20170301,20170601,20170901,
             20180101,20180301,20180601,20180901,
             20190101,20190301,20190601,20190901,
             20200101,20200301,20200601,20200901,
             20210101,20210301,20210601]

def save(start_time,end_time):
    final_df[
        (final_df['timestamp'] >= start_time) & 
        (final_df['timestamp'] < end_time)].to_parquet(
        path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print("Save to ",path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par")
    return None

pool = mp.Pool(20)
t1 = time.time()
for i in tqdm(range(len(time_list)-1)):
    start_time = pd.to_datetime(str(time_list[i]))
    end_time = pd.to_datetime(str(time_list[i+1]))
    pool.apply_async(save, args=(start_time,end_time))  #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 1782.48it/s]


Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-01-01_2015-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2017-01-01_2017-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2019-01-01_2019-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-01-01_2016-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2018-01-01_2018-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-06-01_2015-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2017-06-01_2017-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2019-06-01_2019-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-06-01_2016-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-03-01_2015-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2017-03-01_2017-06-01.par
Save to  /

# Get Factor

In [31]:
day = 30
stride = 10
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0], i] for i in l1[1:]]
    l1 = l1[1:]
    return v + generateC(l1)

def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num, num_rev

def generate_Step_List(data_length, stride):
    if data_length % stride == 0:
        step_list = list(range(0, data_length + stride, stride))
    elif data_length % stride <= 5:
        mod = data_length % stride
        step_list = list(range(0, data_length - stride, stride)) + [data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
    return step_list

In [32]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list

100%|██████████| 26/26 [00:00<00:00, 225313.85it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2016-01-01_2016-03-01.par',
 '2017-01-01_2017-03-01.par',
 '2015-06-01_2015-09-01.par',
 '2018-01-01_2018-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2021-01-01_2021-03-01.par',
 '2016-06-01_2016-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2019-01-01_2019-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2021-03-01_2021-06-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-06-01_2017-09-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-09-01_2021-01-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-03-01_2019-06-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-09-01_2020-01-01.par']

In [35]:
# Read Data

def cov(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis=4, keepdims=True)
        spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                keepdims=True)  # (2000, 1, 36, 1, 1)
        l.append(cov)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def corr(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        std1 = sub_data1.std(axis=4, keepdims=True)
        std2 = sub_data2.std(axis=4, keepdims=True)
        std = (std1 * std2).mean(axis=3, keepdims=True)
        l.append(std)
    l = np.squeeze(np.array(l)).transpose(1, 2, 0)
    fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
    
    # read cov
    cov = pd.read_parquet(output_path.replace("Corr","Cov") + name)
    cov = np.array(cov.set_index(['timestamp','ticker']))
    cov = cov.reshape(l.shape)
    # divide
    l = np.divide(cov, l, out=np.zeros_like(cov, dtype=np.float64), where=l!=0) * fct
    return l

def std(x,num,num_rev,step_list):
    """Main Function"""
    std = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        std1 = sub_data1.std(axis=3, keepdims=True)
        std.append(std1)
    return np.squeeze(np.array(std)).transpose(1, 2, 0)

def zscore(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        mean = sub_data1.mean(axis=3, keepdims=True)
        std = sub_data1.std(axis=3, keepdims=True)
        z_score = np.divide(mean, std, out=np.zeros_like(mean, dtype=np.float64), where=std!=0)
        l.append(z_score)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def ret(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        ret = np.divide(sub_data1[:, :, :, -1], sub_data1[:, :, :, 0], out=sub_data1[:, :, :, -1], where=sub_data1[:, :, :, 0]!=0) - 1
        l.append(ret)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def decay(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        time_spread = end - start
        weight = np.arange(1, time_spread + 1)
        weight = weight / (weight.sum())
        sub_data1 = (x[:, :, :, start:end] * weight).sum(axis=3, keepdims=True)
        l.append(sub_data1)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def test(input_path,name,output_path,method = None):
#     print("read" + input_path + name)
    dataframe_list = pd.read_parquet(input_path + name)
    dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
    x = np.array(dataframe_list.drop('target',axis=1))
    x = x.reshape(x.shape[0], 1, day, -1)
    x = x.transpose(0, 1, 3, 2)

    feat_nums = x.shape[2]
    data_length = x.shape[3]
    num, num_rev = generate_Num_and_ReversedNum(feat_nums)
    step_list = generate_Step_List(data_length, stride)

    """Main Function"""
    if method == "Cov":
        l = cov(x,num,num_rev,step_list)
    if method == "Corr":
        l = cov(x,num,num_rev,step_list)
    if method == "Std":
        l = std(x,num,num_rev,step_list)
    if method == "Zscore":
        l = zscore(x,num,num_rev,step_list)
    if method == "Return":
        l = ret(x,num,num_rev,step_list)
    if method == "Decay":
        l = decay(x,num,num_rev,step_list)

    l = pd.DataFrame(l.reshape(l.shape[0],-1))
    l.index = dataframe_list.index
    l.columns = [str(i) for i in range(l.shape[1])]
    l.reset_index(inplace=True)
    l.to_parquet(output_path + name)
    print("save to ",output_path + name)
    return None

In [38]:
import multiprocessing as mp
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Std','Zscore','Return','Decay']
for i in dir_list:
    for alpha in path_list:
        output = path + i + "/" 
        pool.apply_async(test, args=(data_path,alpha,output,i))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

pool = mp.Pool(10)
for alpha in path_list:
    output = path + "Corr" + "/" 
    pool.apply_async(test, args=(data_path,alpha,output,"Corr"))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-01-01_2016-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-01-01_2015-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2018-01-01_2018-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-09-01_2016-01-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-03-01_2015-06-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-03-01_2016-06-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-06-01_2016-09-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-06-01_2015-09-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2021-01-01_2021-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2017-01-01_2017-03-01.par











  0%|          | 0/3 [00:00<?, ?it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Cov/2015-01-01_2015-03-01.par
read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2020-01-01_2020-03-01.par


 67%|██████▋   | 2/3 [00:11<00:05,  5.74s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Cov/2017-01-01_2017-03-01.par
read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2019-01-01_2019-03-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Cov/2016-01-01_2016-03-01.par
read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2017-03-01_2017-06-01.par


...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2017-09-01_2018-01-01.par


100%|██████████| 3/3 [00:24<00:00,  8.22s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2019-03-01_2019-06-01.par


100%|██████████| 3/3 [00:25<00:00,  8.57s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2020-09-01_2021-01-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2018-09-01_2019-01-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2019-09-01_2020-01-01.par
并行执行时间： 213


# Feat Concat ( Save to Feat_Concat file)

In [1]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
Alpha_Name = "AlphaNet_Original_Input_12.14"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "5d_ret"

In [8]:
os.mkdir(path+"Feat_Concat")

FileExistsError: [Errno 17] File exists: '/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Feat_Concat'

In [2]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list

100%|██████████| 26/26 [00:00<00:00, 155123.62it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2016-01-01_2016-03-01.par',
 '2017-01-01_2017-03-01.par',
 '2015-06-01_2015-09-01.par',
 '2018-01-01_2018-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2021-01-01_2021-03-01.par',
 '2016-06-01_2016-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2019-01-01_2019-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2021-03-01_2021-06-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-06-01_2017-09-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-09-01_2021-01-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-03-01_2019-06-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-09-01_2020-01-01.par']

In [16]:
import multiprocessing as mp
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Corr','Std','Zscore','Return','Decay']

final = []
def test(alpha):
    feat_cat = []
    for i in dir_list:
        feat_cat.append(pd.read_parquet(path + i + "/" + alpha).set_index(['timestamp','ticker']))
    feat_cat = pd.concat(feat_cat,axis=1)
    feat_cat.columns = [str(i) for i in range(feat_cat.shape[1])]
    feat_cat.reset_index().to_parquet(path+"Feat_Concat/"+alpha)
for alpha in tqdm(path_list):
#     test(alpha)
    pool.apply_async(test, args=(alpha,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 58819.80it/s]


并行执行时间： 35


# Feat Daily Norm ( Save to Feat_Concat_Norm_Daily)

In [17]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
Alpha_Name = "AlphaNet_Original_Input_12.14"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "5d_ret"

In [18]:
os.mkdir(path+"Feat_Concat_Norm_Daily")

In [19]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list

100%|██████████| 26/26 [00:00<00:00, 161319.38it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2016-01-01_2016-03-01.par',
 '2017-01-01_2017-03-01.par',
 '2015-06-01_2015-09-01.par',
 '2018-01-01_2018-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2021-01-01_2021-03-01.par',
 '2016-06-01_2016-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2019-01-01_2019-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2021-03-01_2021-06-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-06-01_2017-09-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-09-01_2021-01-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-03-01_2019-06-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-09-01_2020-01-01.par']

In [22]:
for date in path_list[:1]:
    df = pd.read_parquet(path + "Feat_Concat/" + date).set_index(['timestamp','ticker'])
df

,,0,1,2,3,4,5,6,7,8,9,...,314,315,316,317,318,319,320,321,322,323
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,1047.509016,165.116922,249.851660,924.464829,243.390500,183.194222,926.891342,158.088300,78.887819,957.587687,...,861850.412364,0.138453,-0.376716,0.284385,1.522075,1.245018,0.876160,3.017927,2.468587,1.737222
2015-02-17 09:30:00,1,1284.106922,304.746632,233.319300,1251.132811,325.347890,190.544389,1311.161194,268.878420,78.529422,1262.525551,...,825372.445636,-1.631951,-0.601635,0.240629,1.601496,1.147820,0.839082,3.175407,2.275878,1.663689
2015-02-25 09:30:00,1,1515.538580,320.840611,242.242430,1522.025569,325.437271,214.401980,1539.154187,259.118036,99.405562,1482.690747,...,784033.056545,-1.377793,-0.151133,0.292173,1.574869,1.078356,0.797058,3.122595,2.138158,1.580369
2015-02-26 09:30:00,1,1466.712161,355.378729,244.332118,1546.573639,318.650776,222.730596,1381.349806,297.507667,108.085628,1469.916341,...,727636.666545,-0.375218,-0.482991,0.038896,1.643798,1.006125,0.739729,3.259280,1.994944,1.466684
2015-02-27 09:30:00,1,1271.480107,335.406489,121.012389,1338.104279,252.973056,186.957378,1296.812044,215.993378,108.132489,1324.356569,...,756908.456182,-0.411738,-0.287113,0.381687,1.581575,1.159289,0.769491,3.135918,2.298631,1.525691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-02-16 09:30:00,603998,0.828789,0.422953,0.478456,0.432567,0.670000,0.448060,0.657067,0.370296,0.309842,0.663326,...,11102.043636,-0.232562,-0.605060,0.477369,7.366571,9.550095,4.072662,7.366571,9.550095,4.072662
2015-02-17 09:30:00,603998,0.717016,0.501300,0.517742,0.360442,0.646556,0.496018,0.544506,0.254278,0.382506,0.584292,...,11681.422545,0.175280,-0.519973,0.476547,8.692935,8.322640,4.285205,8.692935,8.322640,4.285205
2015-02-25 09:30:00,603998,0.646518,0.589723,0.620109,0.368793,0.598149,0.603898,0.528344,0.360551,0.515889,0.546396,...,12217.920000,1.280062,-0.056055,0.644393,10.261596,7.895082,4.482020,10.261596,7.895082,4.482020


In [25]:
index = df.index

df = np.array(df)
df = df.reshape(df.shape[0],-1,3)

(12935, 324)

In [31]:
df[0]

array([[ 1.04750902e+03,  1.65116922e+02,  2.49851660e+02],
       [ 9.24464829e+02,  2.43390500e+02,  1.83194222e+02],
       [ 9.26891342e+02,  1.58088300e+02,  7.88878189e+01],
       [ 9.57587687e+02,  1.86292675e+02,  2.29790234e+02],
       [ 1.37068107e+07, -8.91245480e+05,  4.31345384e+06],
       [-1.98829705e+01, -6.59192856e+00,  5.30522444e-02],
       [ 1.39344223e+01, -9.06152222e-01,  4.38510846e+00],
       [ 2.76280152e+01, -1.79640700e+00,  8.69465368e+00],
       [ 8.87325873e+02,  1.75757296e+02,  1.50877611e+02],
       [ 9.95996587e+02,  1.93734324e+02,  8.11814422e+01],
       [ 1.05896939e+03,  2.11033576e+02,  2.06326668e+02],
       [ 1.87056244e+07,  3.77890563e+06,  4.23844474e+06],
       [-1.73888487e+00, -5.89884489e+00,  3.23170376e+00],
       [ 1.90160228e+01,  3.84142404e+00,  4.30883549e+00],
       [ 3.77043375e+01,  7.61720067e+00,  8.54345216e+00],
       [ 8.31918736e+02,  2.03409598e+02,  1.07275600e+02],
       [ 8.46815213e+02,  2.15937542e+02

In [34]:
batch = nn.BatchNorm1d(df.shape[1], affine=True)
bc = batch(torch.from_numpy(df).to(torch.float))
bc[0]

tensor([[-1.5858e-02, -1.9725e-02, -1.9354e-02],
        [-1.6246e-02, -1.9487e-02, -1.9774e-02],
        [-1.5126e-02, -1.8948e-02, -1.9342e-02],
        [-1.5769e-02, -1.9424e-02, -1.9218e-02],
        [ 1.2535e+00, -1.1015e-01,  3.7604e-01],
        [-1.6624e-01, -2.8945e-02,  3.9697e-02],
        [ 7.2831e-02, -2.9561e-02,  6.9460e-03],
        [ 1.3613e-01, -3.5884e-02,  2.5445e-02],
        [-1.6350e-02, -1.9764e-02, -1.9883e-02],
        [-1.6402e-02, -1.9908e-02, -2.0400e-02],
        [-1.6313e-02, -2.0009e-02, -2.0030e-02],
        [ 1.2903e+00,  2.3050e-01,  2.6312e-01],
        [-4.2424e-02, -9.0065e-02,  1.4501e-02],
        [ 7.3027e-02, -1.1999e-02, -9.3796e-03],
        [ 1.4434e-01, -2.4258e-04,  4.2084e-03],
        [-1.6853e-02, -2.0150e-02, -2.0655e-02],
        [-1.6695e-02, -1.9898e-02, -2.0147e-02],
        [ 1.2582e+00, -1.3769e-01,  7.6442e-02],
        [-6.2083e-02,  1.4720e-02,  2.2468e-02],
        [ 7.6347e-02, -2.9135e-02, -1.2952e-02],
        [ 1.4128e-01